# AiiDA Status

In [ ]:
from __future__ import print_function

from IPython.display import display, clear_output
import ipywidgets as ipw
import threading
import time
import subprocess

In [ ]:
def on_restart(b):
    with restart_out:
        clear_output()
        ! verdi daemon stop
        ! verdi daemon start
        ! sleep 1
        clear_output()

restart_out = ipw.Output()
btn_restart = ipw.Button(description="Restart Daemon")
btn_restart.on_click(on_restart)
display(btn_restart,restart_out)

In [ ]:
allstates_check = ipw.Checkbox(description="Show all states", value=False)
daemonlog_check = ipw.Checkbox(description="Show daemon log", value=False)

countdown = ipw.IntProgress(value=0, max=9, description="update in")
display(ipw.HBox([countdown, allstates_check, daemonlog_check]))

info_out = ipw.Output()
info_out.layout.border ="2px solid black"
display(info_out)

def update_status():
    while(True):
        with info_out:
            clear_output()
            ! date
            print("\nverdi daemon status")
            ! verdi daemon status
            cmd = "\nverdi calculation list --past-days 1"
            if allstates_check.value:
                cmd += " --all-states"
            print(cmd)
            ! $cmd
            
            if daemonlog_check.value:
                print("\ngrep -i error aiida_daemon.log")
                ! tail -c 1000000 ~/.aiida/daemon/log/aiida_daemon.log | grep -i error

        for i in reversed(range(10)):
            countdown.value = i
            time.sleep(1)

In [ ]:
# start after everything is setup otherwise widgets might endup in wrong output
threading.Thread(target=update_status).start()